In [1]:
"""
[V6]
Blend 6 Models:
* kibuna-nn-hs-1024-last-train (aka. 2stage-NN, LB: 0.01822)
* fork-of-2heads-looper-super-puper-markpeng-infer (LB: 0.1836)
* deepinsight-efficientnet-v7-b3-infer (LB: 0.01850)
* 503-203-tabnet-with-nonscored-features (LB: 0.01836)
* deepinsight_resnest_lightning_v2_infer (LB: 0.01854)

"""

kernel_mode = False

import os
import numpy as np
import pandas as pd
import time
import random
import math
import datetime
import glob

from numba import njit
from scipy.optimize import minimize, fsolve

import optuna

import warnings
warnings.filterwarnings('ignore')

import gc
gc.enable()

rand_seed = 1120

search_mode = True
run_submit_script = False

# method = "scipy"
method = "optuna"
study_name = "moa_blend_team_v6"
# n_trials = 500
# n_trials = 3000
n_trials = 5000

In [2]:
# !mkdir -p /root/.cache/torch/hub/checkpoints/
# !cp ../input/gen-efficientnet-pretrained/tf_efficientnet_*.pth /root/.cache/torch/hub/checkpoints/
# !cp ../input/deepinsight-resnest-v1-resnest50/*.pth /root/.cache/torch/hub/checkpoints/
# !cp ../input/deepinsight-resnest-v2-resnest50-output/*.pth /root/.cache/torch/hub/checkpoints/
# !ls -la /root/.cache/torch/hub/checkpoints/

In [3]:
# !cp ../input/kaggle-moa-team/scripts/* .
# !ls -la

In [4]:
dataset_folder = "../input/lish-moa" if kernel_mode else "/workspace/Kaggle/MoA/"

# Add your model inference script here
# Tuple Format: (script, oof_filename, output_filename, weight)
model_list = [
    ("2stageNN_with_ns_oldcv.py" if kernel_mode else "../../Github/kaggle_moa_team/scripts/2stageNN_with_ns_oldcv.py",
     "../../Github/kaggle_moa_team/oof/oof_2stageNN_ns_oldcv.npy",
     "submission_2stageNN_with_ns_oldcv_0.01822.csv",
     0.30923325055652684),
    
    ("script_simpleNN_oldcv.py" if kernel_mode else "../../Github/kaggle_moa_team/scripts/script_simpleNN_oldcv.py",
     "../../Github/kaggle_moa_team/oof/oof_script_simpleNN_oldcv.npy",
     "submission_script_simpleNN_oldcv_0.01836.csv",
     0.09831493504786226),
    
    ("fork-of-2heads-looper-super-puper-markpeng-infer.py" if kernel_mode else "../../Github/kaggle_moa_team/scripts/fork-of-2heads-looper-super-puper-markpeng-infer.py",
     "../../Github/kaggle_moa_team/oof/oof_fork-of-2heads-looper-super-puper-markpeng.npy",
     "submission_2heads-looper-super-puper_0.01836.csv",
     0.018966959973949222),
    
    ("deepinsight_efficientnet_lightning_v7_b3_infer.py" if kernel_mode else "../../Github/kaggle_moa_team/scripts/deepinsight_efficientnet_lightning_v7_b3_infer.py",
     "../../Github/kaggle_moa_team/oof/oof_deepinsight_efficientnet_lightning_v7_b3_0.01850.npy",
     "submission_effnet_v7_b3_0.01850.csv",
     0.19863369862866234),
    
    ("script_tabnet_ns_oldcv.py" if kernel_mode else "../../Github/kaggle_moa_team/scripts/script_tabnet_ns_oldcv.py",
     "../../Github/kaggle_moa_team/oof/oof_tabnet_ns_oldcv.npy",
     "submission_tabnet_ns_oldcv_0.01836.csv",
     0.0013224625996093413),
    
    ("deepinsight_resnest_lightning_v2_infer.py" if kernel_mode else "../../Github/kaggle_moa_team/scripts/deepinsight_resnest_lightning_v2_infer.py",
     "../../Github/kaggle_moa_team/oof/oof_deepinsight_ResNeSt_v2_resnest50_0.01854.npy",
     "submission_resnest_v2_0.01854.csv",
     0.3728865483320761),
]

model_path = "." if kernel_mode else dataset_folder

In [5]:
train_features = pd.read_csv(f"{dataset_folder}/train_features.csv",
                             engine='c')
train_labels = pd.read_csv(f'{dataset_folder}/train_targets_scored.csv',
                           engine='c')
train_classes = [c for c in train_labels.columns if c != "sig_id"]

non_control_group_rows = train_features["cp_type"] == "trt_cp"
non_control_group_train_labels = train_labels.loc[
    non_control_group_rows, :].copy().reset_index(drop=True)

submission = pd.read_csv(f'{dataset_folder}/sample_submission.csv')
submission.iloc[:, 1:] = 0

In [6]:
def mean_logloss(y_pred, y_true):
    logloss = (1 - y_true) * np.log(1 - y_pred +
                                    1e-15) + y_true * np.log(y_pred + 1e-15)
    return np.nanmean(-logloss)

## Bayesian Optimization and Sequential Least Squares Programming (SLSQP)
https://optuna.readthedocs.io/en/stable/reference/generated/optuna.samplers.TPESampler.html#optuna.samplers.TPESampler

https://docs.scipy.org/doc/scipy/reference/optimize.minimize-slsqp.html

In [7]:
def run_inference_scripts(submission, weights=None):
    for i, (script, oof_filename, output_filename, weight) in enumerate(model_list):
        print(f"Generating submission file from {script} ......")
        infer_start = time.time()
        !python {model_path}/{script}
        infer_elapsed = time.time() - infer_start
        print(f"Time spent on inference: {infer_elapsed/60:.2f} minutes.")

        model_submit = pd.read_csv(output_filename, engine='c')
        print(model_submit.head(5))
        print(model_submit.shape)
        if weights is None:
            print(f"Blending {script} with weight: {weight} ......")
            submission.iloc[:, 1:] += weight * model_submit.iloc[:, 1:]
        else:
            print(f"Blending {script} with weight: {weights[i]} ......")
            submission.iloc[:, 1:] += weights[i] * model_submit.iloc[:, 1:]

    return submission

In [8]:
total_start = time.time()

if not search_mode:
    submission = run_inference_scripts(submission)
elif search_mode and method == "optuna":
    print("[Optuna]")
    ## Search Best Blend Weights by Optuna ##
    model_oofs = []

    for i, (script, oof_filename, output_filename,
            weight) in enumerate(model_list):
        print(f"Loading OOF from {oof_filename} ......")
        oof = np.load(f"{dataset_folder}/{oof_filename}")

        if oof.shape[0] == 23814:
            oof = oof[non_control_group_rows, :]

        oof_loss = mean_logloss(
            oof, non_control_group_train_labels[train_classes].values)
        print(f"OOF Validation Loss of {script}: {oof_loss:.6f}\n")
        model_oofs.append(oof)

    def objective(trial):
        weights = []
        for i in range(len(model_list)):
            weights.append(trial.suggest_float(f"w{i}", 0, 1.0))

        blend = np.zeros(model_oofs[0].shape)
        for i in range(len(model_list)):
            blend += weights[i] * model_oofs[i]
        blend = np.clip(blend, 0, 1.0)

        loss = mean_logloss(
            blend, non_control_group_train_labels[train_classes].values)
        return loss

    pruner = optuna.pruners.MedianPruner(
        n_startup_trials=5,
        n_warmup_steps=0,
        interval_steps=1,
    )
    sampler = optuna.samplers.TPESampler(seed=rand_seed)
    study = optuna.create_study(direction="minimize",
                                pruner=pruner,
                                sampler=sampler,
                                study_name=study_name,
                                storage=f'sqlite:///{study_name}.db',
                                load_if_exists=True)

    study.optimize(objective,
                   n_trials=n_trials,
                   timeout=None,
                   gc_after_trial=True,
                   n_jobs=-1)

    trial = study.best_trial

    if run_submit_script:
        optimal_weights = []
        for i, (script, oof_filename, output_filename, _) in enumerate(model_list):
            optimal_weights.append(trial.params[f"w{i}"])
        submission = run_inference_scripts(submission, weights=optimal_weights)

    print("\n[Optuna]")
    print("Number of finished trials: {}".format(len(study.trials)))
    print("Best trial:")
    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

elif search_mode and method == "scipy":
    print("[Scipy SLSQP]")
    # Optimise Blending Weights with Bonus
    # https://www.kaggle.com/gogo827jz/optimise-blending-weights-with-bonus-0/notebook
    model_oofs = []
    y_true = non_control_group_train_labels[train_classes].values

    all_oof = np.zeros(
        (len(model_list), non_control_group_train_labels.shape[0], 206))
    print(all_oof.shape)
    for i, (script, oof_filename, output_filename,
            weight) in enumerate(model_list):
        print(f"Loading OOF from {oof_filename} ......")
        oof = np.load(f"{dataset_folder}/{oof_filename}")

        if oof.shape[0] == 23814:
            oof = oof[non_control_group_rows, :]

        all_oof[i, :, :] = oof

        oof_loss = mean_logloss(oof, y_true)
        print(f"OOF Validation Loss of {script}: {oof_loss:.6f}\n")
        model_oofs.append(oof)

    # Reference: https://www.kaggle.com/gogo827jz/optimise-blending-weights-with-bonus-0/notebook
    # CPMP's logloss from https://www.kaggle.com/c/lish-moa/discussion/183010
    def log_loss_numpy(y_pred, y_true):
        y_true_ravel = np.asarray(y_true).ravel()
        y_pred = np.asarray(y_pred).ravel()
        y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)
        loss = np.where(y_true_ravel == 1, -np.log(y_pred),
                        -np.log(1 - y_pred))
        return loss.mean()

    def func_numpy_metric(weights):
        oof_blend = np.tensordot(weights, all_oof, axes=((0), (0)))
        return log_loss_numpy(oof_blend, y_true)

    @njit
    def grad_func_jit(weights):
        oof_clip = np.minimum(1 - 1e-15, np.maximum(all_oof, 1e-15))
        gradients = np.zeros(all_oof.shape[0])
        for i in range(all_oof.shape[0]):
            a, b, c = y_true, oof_clip[i], np.zeros(
                (all_oof.shape[1], all_oof.shape[2]))
            for j in range(oof.shape[0]):
                if j != i:
                    c += weights[j] * oof_clip[j]
            gradients[i] = -np.mean(
                (-a * b + (b**2) * weights[i] + b * c) /
                ((b**2) *
                 (weights[i]**2) + 2 * b * c * weights[i] - b * weights[i] +
                 (c**2) - c))
        return gradients

    tol = 1e-10
    init_guess = [1 / all_oof.shape[0]] * all_oof.shape[0]
    bnds = [(0, 1) for _ in range(all_oof.shape[0])]
    cons = {
        'type': 'eq',
        'fun': lambda x: np.sum(x) - 1,
        'jac': lambda x: [1] * len(x)
    }

    print('Inital Blend OOF:', func_numpy_metric(init_guess))

    start_time = time.time()
    res_scipy = minimize(
        fun=func_numpy_metric,
        x0=init_guess,
        method='SLSQP',
        # jac=grad_func_jit,  # grad_func
        bounds=bnds,
        constraints=cons,
        tol=tol)
    print("\n[Scipy SLSQP]")
    print(
        f'[{str(datetime.timedelta(seconds = time.time() - start_time))[2:7]}] Optimised Blend OOF:',
        res_scipy.fun)
    print(f'Optimised Weights: {res_scipy.x}\n')

    if run_submit_script:
        submission = run_inference_scripts(submission, weights=res_scipy.x)

[Optuna]
Loading OOF from ../../Github/kaggle_moa_team/oof/oof_2stageNN_ns_oldcv.npy ......
OOF Validation Loss of ../../Github/kaggle_moa_team/scripts/2stageNN_with_ns_oldcv.py: 0.015606

Loading OOF from ../../Github/kaggle_moa_team/oof/oof_script_simpleNN_oldcv.npy ......
OOF Validation Loss of ../../Github/kaggle_moa_team/scripts/script_simpleNN_oldcv.py: 0.015846

Loading OOF from ../../Github/kaggle_moa_team/oof/oof_fork-of-2heads-looper-super-puper-markpeng.npy ......
OOF Validation Loss of ../../Github/kaggle_moa_team/scripts/fork-of-2heads-looper-super-puper-markpeng-infer.py: 0.015887

Loading OOF from ../../Github/kaggle_moa_team/oof/oof_deepinsight_efficientnet_lightning_v7_b3_0.01850.npy ......
OOF Validation Loss of ../../Github/kaggle_moa_team/scripts/deepinsight_efficientnet_lightning_v7_b3_infer.py: 0.016016

Loading OOF from ../../Github/kaggle_moa_team/oof/oof_tabnet_ns_oldcv.npy ......
OOF Validation Loss of ../../Github/kaggle_moa_team/scripts/script_tabnet_ns_oldc

[I 2020-11-29 00:07:29,472] A new study created in RDB with name: moa_blend_team_v6


OOF Validation Loss of ../../Github/kaggle_moa_team/scripts/deepinsight_resnest_lightning_v2_infer.py: 0.015756



[I 2020-11-29 00:07:30,441] Trial 0 finished with value: 0.024601614675104282 and parameters: {'w0': 0.8344250470071957, 'w1': 0.20969926246480586, 'w2': 0.33693279072812676, 'w3': 0.4743904818510878, 'w4': 0.9092365194919435, 'w5': 0.5243087249499285}. Best is trial 0 with value: 0.016543749579780553.
[I 2020-11-29 00:07:30,458] Trial 3 finished with value: 0.02700148311904691 and parameters: {'w0': 0.7365631164327696, 'w1': 0.8208881819897237, 'w2': 0.3691996396938644, 'w3': 0.5124905762258, 'w4': 0.5621715420755135, 'w5': 0.7491883465011567}. Best is trial 0 with value: 0.016543749579780553.
[I 2020-11-29 00:07:30,500] Trial 1 finished with value: 0.016543749579780553 and parameters: {'w0': 0.3811516322674158, 'w1': 0.20005692424919308, 'w2': 0.15126262473036878, 'w3': 0.20003903173061044, 'w4': 0.35791608336555947, 'w5': 0.12432271229682545}. Best is trial 1 with value: 0.016543749579780553.
[I 2020-11-29 00:07:30,504] Trial 2 finished with value: 0.022888287073748438 and parameter

[I 2020-11-29 00:07:34,038] Trial 26 finished with value: 0.017434702676333125 and parameters: {'w0': 0.6589429135799105, 'w1': 0.2459643447839675, 'w2': 0.020006016078040034, 'w3': 0.2777176803483442, 'w4': 0.38755222711330184, 'w5': 0.011250502356814351}. Best is trial 24 with value: 0.016129333675918657.
[I 2020-11-29 00:07:34,308] Trial 30 finished with value: 0.017418172372321968 and parameters: {'w0': 0.6512939364956676, 'w1': 0.2420847328608715, 'w2': 0.03252172954764158, 'w3': 0.2828610986411347, 'w4': 0.3825226122789761, 'w5': 0.0015726004138083838}. Best is trial 24 with value: 0.016129333675918657.
[I 2020-11-29 00:07:34,326] Trial 28 finished with value: 0.016596441448266382 and parameters: {'w0': 0.6314309837114156, 'w1': 0.08593531010747792, 'w2': 0.007980487981396795, 'w3': 0.2853612329613892, 'w4': 0.3556649044301262, 'w5': 0.02922783700243635}. Best is trial 24 with value: 0.016129333675918657.
[I 2020-11-29 00:07:34,338] Trial 31 finished with value: 0.016205260476721

...

[I 2020-11-29 00:25:56,329] Trial 4913 finished with value: 0.01518258951815535 and parameters: {'w0': 0.25916077831042167, 'w1': 0.1272770564997326, 'w2': 0.032165627856848795, 'w3': 0.19717098964061394, 'w4': 0.0007637876111470615, 'w5': 0.35694602091201194}. Best is trial 2855 with value: 0.015173437622007157.
[I 2020-11-29 00:25:56,337] Trial 4910 finished with value: 0.015244852749596 and parameters: {'w0': 0.2685744425520249, 'w1': 0.16668250958728936, 'w2': 0.033969074628242675, 'w3': 0.19498217435498094, 'w4': 0.01702684479572682, 'w5': 0.3618594510912558}. Best is trial 2855 with value: 0.015173437622007157.
[I 2020-11-29 00:25:56,340] Trial 4912 finished with value: 0.01540144162373962 and parameters: {'w0': 0.2685342550179938, 'w1': 0.1296240278734784, 'w2': 0.00022765882128754703, 'w3': 0.02537401986343532, 'w4': 0.0004566336575122203, 'w5': 0.3540119572631167}. Best is trial 2855 with value: 0.015173437622007157.
[I 2020-11-29 00:25:56,388] Trial 4914 finished with value: 

[I 2020-11-29 00:26:04,730] Trial 4938 finished with value: 0.015465292011477574 and parameters: {'w0': 0.3167499371117107, 'w1': 0.07489115469791677, 'w2': 0.0851605859375891, 'w3': 0.21093752883275524, 'w4': 0.0184929298063633, 'w5': 0.41928925100267356}. Best is trial 2855 with value: 0.015173437622007157.
[I 2020-11-29 00:26:05,028] Trial 4934 finished with value: 0.015466528017038373 and parameters: {'w0': 0.3173809399744709, 'w1': 0.06826384845514509, 'w2': 0.09011428213431366, 'w3': 0.21251266140727873, 'w4': 0.018646852772932213, 'w5': 0.4190375021275521}. Best is trial 2855 with value: 0.015173437622007157.
[I 2020-11-29 00:26:06,042] Trial 4941 finished with value: 0.01519405199263139 and parameters: {'w0': 0.2455436771778535, 'w1': 0.14561558308377404, 'w2': 0.0003635705205876801, 'w3': 0.22589750559362598, 'w4': 0.0008199062640219577, 'w5': 0.40066862368932926}. Best is trial 2855 with value: 0.015173437622007157.
[I 2020-11-29 00:26:06,437] Trial 4939 finished with value: 

[I 2020-11-29 00:26:12,151] Trial 4963 finished with value: 0.01530038287849065 and parameters: {'w0': 0.33623353676074585, 'w1': 0.057836193953076355, 'w2': 0.05286067558776101, 'w3': 0.20172051134553257, 'w4': 0.01852711316341078, 'w5': 0.40106744714196735}. Best is trial 2855 with value: 0.015173437622007157.
[I 2020-11-29 00:26:13,538] Trial 4965 finished with value: 0.01526279083700133 and parameters: {'w0': 0.3137531852725893, 'w1': 0.11750212610948192, 'w2': 0.01756215226072229, 'w3': 0.1738161617928863, 'w4': 0.05158670024855407, 'w5': 0.3797252742657032}. Best is trial 2855 with value: 0.015173437622007157.
[I 2020-11-29 00:26:13,940] Trial 4966 finished with value: 0.015260605031491839 and parameters: {'w0': 0.3123152115058882, 'w1': 0.11661558692004342, 'w2': 0.01742301650440881, 'w3': 0.17340481144705686, 'w4': 0.0504547037733985, 'w5': 0.3810549399766005}. Best is trial 2855 with value: 0.015173437622007157.
[I 2020-11-29 00:26:13,941] Trial 4969 finished with value: 0.017

[I 2020-11-29 00:26:21,113] Trial 4993 finished with value: 0.016962021568084983 and parameters: {'w0': 0.26142928837850116, 'w1': 0.046819086085162526, 'w2': 0.03699471226746493, 'w3': 0.21276054088418192, 'w4': 0.6105350896469409, 'w5': 0.38784745468043164}. Best is trial 2855 with value: 0.015173437622007157.
[I 2020-11-29 00:26:21,143] Trial 4991 finished with value: 0.015204500064831218 and parameters: {'w0': 0.25288984493371225, 'w1': 0.04722788053497273, 'w2': 0.0003470731185021836, 'w3': 0.2152311746844336, 'w4': 0.03179666033339546, 'w5': 0.3925023307766744}. Best is trial 2855 with value: 0.015173437622007157.
[I 2020-11-29 00:26:21,148] Trial 4994 finished with value: 0.018131097062961184 and parameters: {'w0': 0.25601894816320997, 'w1': 0.05120664505146253, 'w2': 0.04629692796725701, 'w3': 0.21820184397938733, 'w4': 0.8902622262078916, 'w5': 0.3898843905003583}. Best is trial 2855 with value: 0.015173437622007157.
[I 2020-11-29 00:26:21,153] Trial 4992 finished with value: 


[Optuna]
Number of finished trials: 5000
Best trial:
  Value: 0.015173437622007157
  Params: 
    w0: 0.30923325055652684
    w1: 0.09831493504786226
    w2: 0.018966959973949222
    w3: 0.19863369862866234
    w4: 0.0013224625996093413
    w5: 0.3728865483320761


In [9]:
total_elapsed = time.time() - total_start
print(f"Total time spent: {total_elapsed/60:.2f} minutes.")

Total time spent: 18.91 minutes.


In [10]:
# [V6]
# [Optuna]
# Number of finished trials: 5000
# Best trial:
#   Value: 0.015173437622007157
#   Params: 
#     w0: 0.30923325055652684
#     w1: 0.09831493504786226
#     w2: 0.018966959973949222
#     w3: 0.19863369862866234
#     w4: 0.0013224625996093413
#     w5: 0.3728865483320761

# [Scipy SLSQP]
# [00:36] Optimised Blend OOF: 0.015172005464591968
# Optimised Weights: [3.20472642e-01 9.01191588e-02 1.78893358e-18 2.20448482e-01
#  3.27971157e-18 3.68959717e-01]

In [11]:
# [V5]
# Number of finished trials: 3000
# Best trial:
#   Value: 0.015344701181290615
#   Params: 
#     w0: 0.5141433844379889
#     w1: 0.11747776562133813
#     w2: 0.3668324643717302

# [00:14] Optimised Blend OOF: 0.015344695215068541
# Optimised Weights: [0.51922623 0.11292509 0.36784869]

In [12]:
# [V4]
# [Optuna]
# Number of finished trials: 3000
# Best trial:
#   Value: 0.015331901615194453
#   Params: 
#     w0: 0.4505928450756189
#     w1: 0.13010257032841785
#     w2: 0.06308933354044946
#     w3: 0.35639153615958885
#
# [Scipy]
# [00:23] Optimised Blend OOF: 0.015331777381591449
# Optimised Weights: [0.44090106 0.14508641 0.05945655 0.35455598]

In [13]:
# [V3]
# improving-mark-s-2-heads-model-infer
# Number of finished trials: 3000
# Best trial:
#   Value: 0.01515466145873492
#   Params: 
#     w0: 0.0002980690037490555
#     w1: 0.29771381784976886
#     w2: 0.1569191862042946
#     w3: 0.18156875605872544
#     w4: 0.36371774630338105

In [14]:
# [V3]
# fork-of-2heads-looper-super-puper-markpeng-infer
# Number of finished trials: 3000
# Best trial:
#   Value: 0.015170138066049686
#   Params: 
#     w0: 0.00019903389488299251
#     w1: 0.3853752127955825
#     w2: 0.015968332256452233
#     w3: 0.22945916769823432
#     w4: 0.3711290150522236

In [15]:
if run_submit_script:
    print(submission.shape)
    print(submission)
    submission.to_csv('submission.csv', index=False)

## EOF

In [16]:
if kernel_mode:
    !rm ./*.py
    !ls -la